## Importing Libraries

In [1]:
import pandas as pd
import os

In [2]:
par = pd.read_parquet('extra_data.parquet')
par.head()

,index,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
1,1,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
2,2,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
3,3,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0
4,4,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0


In [3]:
grocery_sales = pd.read_csv('grocery_sales.csv')
grocery_sales.head()

,index,Store_ID,Date,Dept,Weekly_Sales
0,0,1,2010-02-05,1,24924.50
1,1,1,2010-02-05,26,11737.12
2,2,1,2010-02-05,17,13223.76
3,3,1,2010-02-05,45,37.44
4,4,1,2010-02-05,28,1085.29


In [151]:
def extract(df,parq):
    par_df = pd.read_parquet(parq)
    merged_df = df.merge(par_df, left_on = 'index', right_on = 'index')
    return merged_df

In [152]:
merged_df = extract(grocery_sales,'extra_data.parquet')
merged_df.head()

,index,Store_ID,Date,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,2010-02-05,1,24924.50,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
1,1,1,2010-02-05,26,11737.12,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
2,2,1,2010-02-05,17,13223.76,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
3,3,1,2010-02-05,45,37.44,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0
4,4,1,2010-02-05,28,1085.29,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0


In [153]:
merged_df.isna().sum()

index            0
Store_ID         0
Date            39
Dept             0
Weekly_Sales    38
IsHoliday        0
Temperature      0
Fuel_Price       0
MarkDown1        0
MarkDown2        0
MarkDown3        0
MarkDown4        1
MarkDown5        1
CPI             47
Unemployment    37
Type             1
Size             1
dtype: int64

In [154]:
def transform(df):
    df.fillna({
        'Weekly_Sales':df['Weekly_Sales'].mean(),
        'MarkDown4':df['MarkDown4'].mode(),
        'MarkDown5':df['MarkDown5'].mode(),
        'CPI':df['CPI'].mean(),
        'Unemployment':df['Unemployment'].mean(),
        'Type':df['Type'].mode(),
        'Size':df['Size'].mode(),
    }, inplace= True)
    df.fillna(method = 'ffill', inplace=True)
    
    df['Month'] = df['Date'].dt.month
    
    df = df[df['Weekly_Sales'] > 10000]
    
    df = df[["Store_ID","Month","Dept","IsHoliday","Weekly_Sales","CPI","Unemployment"]]

    return df

In [155]:
clean_data = transform(merged_df)
clean_data.head()

,Store_ID,Month,Dept,IsHoliday,Weekly_Sales,CPI,Unemployment
0,1,2,1,0,24924.50,211.096358,8.106000
1,1,2,26,0,11737.12,211.096358,8.106000
2,1,2,17,0,13223.76,211.096358,8.106000
5,1,2,79,0,46729.77,211.096358,7.500052
6,1,2,55,0,21249.31,211.096358,7.500052


In [156]:
clean_data.isna().sum()

Store_ID        0
Month           0
Dept            0
IsHoliday       0
Weekly_Sales    0
CPI             0
Unemployment    0
dtype: int64

In [157]:
def avg_monthly_sales(df):
    df = pd.DataFrame(data = df.groupby('Month').Weekly_Sales.agg('mean'))
    df.reset_index(inplace=True)
    df.columns = ['Month','Avg_Sales']
    df['Avg_Sales'] = df['Avg_Sales'].round(decimals = 2)
    return df

In [158]:
agg_data = avg_monthly_sales(clean_df)
agg_data.head(13)

,Month,Avg_Sales
0,1.0,33174.18
1,2.0,34333.33
2,3.0,33220.89
3,4.0,33392.37
4,5.0,33339.89
5,6.0,34582.47
6,7.0,33922.76
7,8.0,33644.79
8,9.0,33258.05
9,10.0,32736.99


In [159]:
def load(df1,df2,path1,path2):
    df1.to_csv(path1, index=False)
    df2.to_csv(path2, index=False)

In [160]:
load(clean_data,agg_data,'clean_data.csv','agg_data.csv')

In [161]:
def validation(path1):
    if os.path.exists(path1) == 0:
        raise Exception('Path 1 does not exists.')


In [162]:
validation('clean_data.csv')